Подключим необходимые библиотеки

In [2]:
%pylab inline

import numpy as np
import pandas as pd
import matplotlib as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

#import xgboost as xgb

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


Считаем файлы с данными

In [3]:
dtrain = pd.read_csv('Xtrain.csv')
dtest = pd.read_csv('Xtest.csv')

dytrain = pd.read_csv('ytrain.csv')

C:\Python27\lib\site-packages\pandas\io\parsers.py:1159: DtypeWarning: Columns (328,330,332,335,337,343,344,345,347,354,355,356,357,360,361,363,366,371,376,379,382,384,386,389,391,398,399,407,408,415,417,418,432,436,437,440,441,447,448,449,452,456,463,477,479,480,481,482,484,485,488,495,496,497,498,499,502,506,507,508,509,513,514,516,518,519,520,521,522,526,529,533,534,538,540,543,547,548,549,550,557,559,568,571,572,575,578,582,595,596,597,599,600,601,604,605,608,612,618,619,624,626,627,630,631,635,640,641,642,643,644,645,647,648,650,654,660,661,664,674,675,680,681,684,686,687,689,690,691,693,694,697,700,701,702,705,710,711,712,718,719,720,732,734,735,736,737,739,741,745,746,747,748,750,751,753,754,759,763,767,769,780,781,788,792,796,806,811,813,816,821,822,823,824,831,839,842,843,844,849,850,852,860,866,867,872,873,875,878,879,885,889,892,898,899,900,901,903,904,907,908,909,912,913,921,922,930,932,934,938,941,942,945,950,954,959,963,964,967,968,969,972,976,985,986,993,995,1007,1013,10

Создадим матрицу с известными значениями результатов

In [4]:
ytrain = np.array(dytrain.ix[:, 1:])

Соединим вместе трейн и тест, заменим пропуски на -999

In [5]:
#dall = pd.concat((dtrain, dtest))
#dtrain.fillna(-999., inplace = True)
#dtest.fillna(-999., inplace=True)

Категориальные фичи перекодируем в числа

In [6]:
from sklearn.preprocessing import Imputer


Xtrain = np.array(dtrain.ix[:, 1:])
Xtest = np.array(dtest.ix[:, 1:])


for i in range(327, 1378):
    Xtrain[:, i] = LabelEncoder().fit_transform(Xtrain[:, i])
    Xtest[:, i] = LabelEncoder().fit_transform(Xtest[:, i])


C:\Python27\lib\site-packages\numpy\lib\arraysetops.py:198: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [7]:
imputer = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imputer.fit(Xtrain)
Xtrain = imputer.transform(Xtrain)
imputer.fit(Xtest)
Xtest = imputer.transform(Xtest)

In [8]:
Xtrain[:10,:10]

array([[ 0.07692308,  0.9047619 ,  0.02544943,  0.04761905,  0.16666667,
         0.36842105,  0.        ,  0.        ,  0.        ,  0.04979253],
       [ 0.07692308,  0.9047619 ,  0.02447477,  0.04761905,  0.16666667,
         0.34210526,  0.        ,  0.        ,  0.        ,  0.04979253],
       [ 0.07692308,  0.9047619 ,  0.04169374,  0.04761905,  0.16666667,
         0.44736842,  0.        ,  0.        ,  0.        ,  0.04979253],
       [ 0.07692308,  0.9047619 ,  0.03941954,  0.04761905,  0.16666667,
         0.36842105,  0.        ,  0.        ,  0.        ,  0.04979253],
       [ 0.07692308,  0.9047619 ,  0.04916613,  0.04761905,  0.16666667,
         0.71052632,  0.        ,  0.        ,  0.        ,  0.04979253],
       [ 0.07692308,  0.9047619 ,  0.05024908,  0.04761905,  0.16666667,
         0.39473684,  0.        ,  0.        ,  0.        ,  0.04979253],
       [ 0.07692308,  0.9047619 ,  0.04949101,  0.04761905,  0.16666667,
         0.5       ,  0.        ,  0.        

Кросс-валидация:

In [10]:
from sklearn.cross_validation import KFold

cv = KFold(n=len(Xtrain), n_folds=5, shuffle=True, random_state=49)

In [14]:
print Xtrain.shape, ytrain.shape

(10000, 1376) (10000, 14)


In [ ]:
from sklearn import grid_search
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import BaggingRegressor as BR
from sklearn.tree import DecisionTreeRegressor as DT


ypred = np.zeros((len(Xtest), 14))
for i in range(14):
    #params = {'n_estimators': [100, 200, 300], 'max_depth': [10, 20, 30]}
    #grid_searcher = grid_search.GridSearchCV(GBR(learning_rate=0.1, loss='ls'), params, cv=cv, scoring='mean_squared_error')

    gbr = GBR(learning_rate=0.1, max_depth=10, n_estimators=100, loss='ls')
    gbr.fit(Xtrain, ytrain[:,i])
    br = BR(DT(), n_estimators = 100, n_jobs=8)
    br.fit(Xtrain, ytrain[:,i])
    print i, mse(gbr.predict(Xtrain)[:, i],  ytrain[:,i]), mse(bt.predict(Xtrain)[:, i],  ytrain[:,i])
    #ypred = grid_searcher.predict_proba(Xtest)[:, 1]

    #print grid_searcher.best_score_
    #print grid_searcher.best_estimator_

Настроим параметры бустинга

In [6]:
param = {}
param['objective'] = 'binary:logistic'
param['eta'] = 0.1
param['subsample'] = 0.8
param['colsample_bytree'] = 0.6
param['silent'] = 1
param['max_depth'] = 6
param['seed'] = 0
param['num_parallel_tree'] = 1
param['nthread'] = 30

numround = 50

plst = list(param.items())

In [7]:
del dtrain
del dtest

Для каждой колонки обучим и предскажем значения

In [9]:
ypred = np.zeros((len(Xtest), 14))

for i in range(14):
    Xgbtrain = xgb.DMatrix(data = Xtrain, label = ytrain[:, i])
    Xgbtest = xgb.DMatrix(data = Xtest)
    
    bst = xgb.train(plst, Xgbtrain, numround)
    
    ypred[:, i] = bst.predict(Xgbtest)
    
    print i

0
1
2
3
4
5
6
7
8
9
10
11
12
13


Создадим файл с предсказаниями и сохраним его

In [13]:
dresult = pd.DataFrame()

dresult['id'] = dtest['id']

for i in range(14):
    dresult['y' + str(i)] = ypred[:, i]
    
dresult.to_csv('result.csv', index = False)

In [ ]:
from sklearn.metrics import mean_squared_error as mse

print mse(ypred)